In [1]:
import os
import pandas as pd
import numpy as np
from src.feature_engineering import CLTVFeatureEngineering

In [2]:
file_path = '/home/kabyik/Desktop/customer-lifecycle-management/data/Online_Retail.xlsx'
if os.path.exists(file_path):
    df = pd.read_excel(file_path)
else:
    print(f"File not found: {file_path}")

In [3]:
from src.drop_columns import ColumnDropper
from src.handle_missing_values import DropMissingValues
from src.outlier_detection import OutlierDetector


In [4]:
df_copy = df.copy()

In [5]:
df_copy.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
dropper = ColumnDropper(['InvoiceNo', 'StockCode','Description', 'Country'])
df_dropped = dropper.drop_columns(df_copy)

2024-12-07 15:41:30,386 - INFO - Attempting to drop columns: ['InvoiceNo', 'StockCode', 'Description', 'Country']
2024-12-07 15:41:30,392 - INFO - Columns successfully dropped: ['InvoiceNo', 'StockCode', 'Description', 'Country']


In [7]:
df_dropped.head()

,Quantity,InvoiceDate,UnitPrice,CustomerID
0,6,2010-12-01 08:26:00,2.55,17850.0
1,6,2010-12-01 08:26:00,3.39,17850.0
2,8,2010-12-01 08:26:00,2.75,17850.0
3,6,2010-12-01 08:26:00,3.39,17850.0
4,6,2010-12-01 08:26:00,3.39,17850.0


In [8]:
df_dropped["Amount"] = df_dropped["Quantity"] * df_dropped["UnitPrice"]

In [9]:
df_transformed = df_dropped.copy()
df_transformed['InvoiceDate'] = pd.to_datetime(df_transformed['InvoiceDate'])

In [10]:
# Group by CustomerID and aggregate the data
df_grouped = df_dropped.groupby('CustomerID').agg({
    'InvoiceDate': ['min','max','count'],
    'Amount': 'sum'
}).reset_index()

print(df_grouped)

     CustomerID         InvoiceDate                             Amount
                                min                 max count      sum
0       12346.0 2011-01-18 10:01:00 2011-01-18 10:17:00     2     0.00
1       12347.0 2010-12-07 14:57:00 2011-12-07 15:52:00   182  4310.00
2       12348.0 2010-12-16 19:09:00 2011-09-25 13:13:00    31  1797.24
3       12349.0 2011-11-21 09:51:00 2011-11-21 09:51:00    73  1757.55
4       12350.0 2011-02-02 16:01:00 2011-02-02 16:01:00    17   334.40
...         ...                 ...                 ...   ...      ...
4367    18280.0 2011-03-07 09:52:00 2011-03-07 09:52:00    10   180.60
4368    18281.0 2011-06-12 10:53:00 2011-06-12 10:53:00     7    80.82
4369    18282.0 2011-08-05 13:35:00 2011-12-02 11:43:00    13   176.60
4370    18283.0 2011-01-06 14:14:00 2011-12-06 12:02:00   756  2094.88
4371    18287.0 2011-05-22 10:39:00 2011-10-28 09:29:00    70  1837.28

[4372 rows x 5 columns]


In [11]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4372 entries, 0 to 4371
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   (CustomerID, )        4372 non-null   float64       
 1   (InvoiceDate, min)    4372 non-null   datetime64[ns]
 2   (InvoiceDate, max)    4372 non-null   datetime64[ns]
 3   (InvoiceDate, count)  4372 non-null   int64         
 4   (Amount, sum)         4372 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(1)
memory usage: 170.9 KB


In [12]:
current_date = df_dropped['InvoiceDate'].max()
print(current_date)

2011-12-09 12:50:00


In [13]:
df_dropped.head()

,Quantity,InvoiceDate,UnitPrice,CustomerID,Amount
0,6,2010-12-01 08:26:00,2.55,17850.0,15.30
1,6,2010-12-01 08:26:00,3.39,17850.0,20.34
2,8,2010-12-01 08:26:00,2.75,17850.0,22.00
3,6,2010-12-01 08:26:00,3.39,17850.0,20.34
4,6,2010-12-01 08:26:00,3.39,17850.0,20.34


In [14]:
from src.feature_engineering import CLTVFeatureEngineering,FeatureEngineer
cltv_fea = FeatureEngineer(CLTVFeatureEngineering('CustomerID','InvoiceDate','Amount'))
df_grouped1 = cltv_fea.apply_feature_engineering(df_dropped)
print(df_grouped1)

2024-12-07 15:41:30,537 - INFO - Applying feature engineering strategy.
2024-12-07 15:41:30,539 - INFO - Creating CLTV features
2024-12-07 15:41:30,637 - INFO - CLTV feature engineering completed


      CustomerID  frequency  total_amount  avg_order_value  recency  \
0        12346.0          2          0.00         0.000000      325   
1        12347.0        182       4310.00        23.681319        1   
2        12348.0         31       1797.24        57.975484       74   
3        12349.0         73       1757.55        24.076027       18   
4        12350.0         17        334.40        19.670588      309   
...          ...        ...           ...              ...      ...   
4367     18280.0         10        180.60        18.060000      277   
4368     18281.0          7         80.82        11.545714      180   
4369     18282.0         13        176.60        13.584615        7   
4370     18283.0        756       2094.88         2.771005        3   
4371     18287.0         70       1837.28        26.246857       42   

      customer_age  lifetime  purchase_frequency      CLTV  
0              325         0            0.043077      0.00  
1              366       

In [15]:
df_grouped1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4369 entries, 0 to 4371
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          4369 non-null   float64
 1   frequency           4369 non-null   int64  
 2   total_amount        4369 non-null   float64
 3   avg_order_value     4369 non-null   float64
 4   recency             4369 non-null   int64  
 5   customer_age        4369 non-null   int64  
 6   lifetime            4369 non-null   int64  
 7   purchase_frequency  4369 non-null   float64
 8   CLTV                4369 non-null   float64
dtypes: float64(5), int64(4)
memory usage: 341.3 KB


In [16]:
print(df_grouped1[df_grouped1['CLTV'].isnull()])

Empty DataFrame
Columns: [CustomerID, frequency, total_amount, avg_order_value, recency, customer_age, lifetime, purchase_frequency, CLTV]
Index: []


In [17]:
df_filtered = df_grouped[df_grouped['CustomerID'].isin([12713.0, 13298.0, 14569.0])]
print(df_filtered)

     CustomerID         InvoiceDate                            Amount
                                min                 max count     sum
301     12713.0 2011-12-09 12:16:00 2011-12-09 12:16:00    38  848.55
723     13298.0 2011-12-08 13:11:00 2011-12-08 13:11:00     2  360.00
1649    14569.0 2011-12-08 14:58:00 2011-12-08 14:58:00    12  227.39


In [18]:
df_grouped1 = df_grouped1.dropna(subset=['CLTV'])
print(df_grouped1)

      CustomerID  frequency  total_amount  avg_order_value  recency  \
0        12346.0          2          0.00         0.000000      325   
1        12347.0        182       4310.00        23.681319        1   
2        12348.0         31       1797.24        57.975484       74   
3        12349.0         73       1757.55        24.076027       18   
4        12350.0         17        334.40        19.670588      309   
...          ...        ...           ...              ...      ...   
4367     18280.0         10        180.60        18.060000      277   
4368     18281.0          7         80.82        11.545714      180   
4369     18282.0         13        176.60        13.584615        7   
4370     18283.0        756       2094.88         2.771005        3   
4371     18287.0         70       1837.28        26.246857       42   

      customer_age  lifetime  purchase_frequency      CLTV  
0              325         0            0.043077      0.00  
1              366       

In [19]:
df_grouped1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4369 entries, 0 to 4371
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          4369 non-null   float64
 1   frequency           4369 non-null   int64  
 2   total_amount        4369 non-null   float64
 3   avg_order_value     4369 non-null   float64
 4   recency             4369 non-null   int64  
 5   customer_age        4369 non-null   int64  
 6   lifetime            4369 non-null   int64  
 7   purchase_frequency  4369 non-null   float64
 8   CLTV                4369 non-null   float64
dtypes: float64(5), int64(4)
memory usage: 341.3 KB
